In [1]:
import pandas as pd
import numpy as np
import random
import math
import os

import matplotlib.pyplot as plt
import statsmodels.api as sm
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model


os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

tf.random.set_seed(3407)  # Replace 42 with your desired seed number
np.random.seed(3407)  # Numpy seed
random.seed(3407)     # Python's built-in random module

In [2]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
  dataX, dataY = [], []
  for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back)]
    dataX.append(a)
    dataY.append(dataset[i + look_back])
  return np.array(dataX), np.array(dataY)

def fill_nan(arr):
    # Forward fill
    mask = np.isnan(arr)
    idx = np.where(~mask, np.arange(mask.shape[0]), 0)
    np.maximum.accumulate(idx, axis=0, out=idx)
    out = arr[idx]

    # Backward fill for the remaining NaNs
    mask = np.isnan(out)
    idx = np.where(~mask, np.arange(mask.shape[0]), mask.shape[0] - 1)
    idx = np.minimum.accumulate(idx[::-1], axis=0)[::-1]
    out = out[idx]

    return out

# split into train and test sets
def split_dataset(array):
  train_size = int(len(array) * 0.67)
  test_size = len(array) - train_size
  train, test = array[:train_size], array[train_size:len(array)]
  return train, test

# load the dataset
dataframe = pd.read_csv('data_daily.csv', usecols=[1], engine='python')

dataset = dataframe.astype('float32')
dataset = dataset.values
dataset = dataset/1e5

# Decompose the time series
decomposition = sm.tsa.seasonal_decompose(dataset, model='additive', period = 12)  # or model='multiplicative' based on your data

# Extract the trend, seasonality, and residuals
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

trend = fill_nan(trend)
seasonal = fill_nan(seasonal)
residual = fill_nan(residual)

trend = trend.reshape(-1,1)
seasonal = seasonal.reshape(-1,1)
residual = residual.reshape(-1,1)

In [21]:
# Defined look back period (days)
look_back = 90

# Define inputs for each aspect
input_dataset = Input(shape=(look_back,))
input_residuals = Input(shape=(look_back,))
input_trend = Input(shape=(look_back,))
input_seasonal = Input(shape=(look_back,))

# Subnetwork for dataset
net_dataset = Dense(12, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(input_dataset)
net_dataset = Dense(8, activation='swish', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(net_dataset)

# Subnetwork for residuals
net_residuals = Dense(12, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(input_residuals)
net_residuals = Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(net_residuals)
net_residuals = Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(net_residuals)

# Subnetwork for trend
net_trend = Dense(12, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(input_trend)
net_trend = Dense(8, activation='swish', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(net_trend)

# Subnetwork for seasonal
net_seasonal = Dense(12, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(input_seasonal)
net_seasonal = Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(net_seasonal)
net_seasonal = Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(net_seasonal)

# Combine all subnetworks
combined = concatenate([net_dataset, net_residuals, net_trend, net_seasonal])

# Separate output layers for each prediction target
output_dataset = Dense(1, name='output_dataset')(combined)
output_residuals = Dense(1, name='output_residuals')(combined)
output_trend = Dense(1, name='output_trend')(combined)
output_seasonal = Dense(1, name='output_seasonal')(combined)

# Create multi-output model
model = Model(inputs=[input_dataset, input_residuals, input_trend, input_seasonal],
              outputs=[output_dataset, output_residuals, output_trend, output_seasonal])
model.compile(optimizer='adam',
              loss={'output_dataset': 'mean_squared_error',
                    'output_residuals': 'mean_squared_error',
                    'output_trend': 'mean_squared_error',
                    'output_seasonal': 'mean_squared_error'},
              metrics={'output_dataset': 'mae',
                       'output_residuals': 'mae',
                       'output_trend': 'mae',
                       'output_seasonal': 'mae'})

In [ ]:
# Prepare data for each aspect
trainX_dataset, trainY_dataset = create_dataset(dataset, look_back)
trainX_residuals, trainY_residuals = create_dataset(residual, look_back)
trainX_trend, trainY_trend = create_dataset(trend, look_back)
trainX_seasonal, trainY_seasonal = create_dataset(seasonal, look_back)

model.fit([trainX_dataset, trainX_residuals, trainX_trend, trainX_seasonal],
          {'output_dataset': trainY_dataset,
           'output_residuals': trainY_residuals,
           'output_trend': trainY_trend,
           'output_seasonal': trainY_seasonal},
          epochs=10, batch_size=32, verbose=2, shuffle = False)

In [ ]:
# Extract the last 'look_back' values from each aspect
last_values_dataset = dataset[-look_back:].reshape(1, look_back)
last_values_residuals = residual[-look_back:].reshape(1, look_back)
last_values_trend = trend[-look_back:].reshape(1, look_back)
last_values_seasonal = seasonal[-look_back:].reshape(1, look_back)

num_future_steps = 365  # Number of future steps to forecast

# Create empty lists to store the forecasted values for each aspect
forecasted_dataset = []
forecasted_residuals = []
forecasted_trend = []
forecasted_seasonal = []

for _ in range(num_future_steps):
    # Predict the next set of values
    predictions = model.predict([last_values_dataset, last_values_residuals, last_values_trend, last_values_seasonal])
    predicted_dataset, predicted_residuals, predicted_trend, predicted_seasonal = predictions

    # Append the predicted values to the forecast lists
    forecasted_dataset.append(predicted_dataset[0, 0])
    forecasted_residuals.append(predicted_residuals[0, 0])
    forecasted_trend.append(predicted_trend[0, 0])
    forecasted_seasonal.append(predicted_seasonal[0, 0])

    # Update the last_values arrays with the newly predicted values
    # For each aspect, we roll the array to remove the first (oldest) value and append the new prediction
    last_values_dataset = np.roll(last_values_dataset, -1)
    last_values_dataset[0, -1] = predicted_dataset

    last_values_residuals = np.roll(last_values_residuals, -1)
    last_values_residuals[0, -1] = predicted_residuals

    last_values_trend = np.roll(last_values_trend, -1)
    last_values_trend[0, -1] = predicted_trend

    last_values_seasonal = np.roll(last_values_seasonal, -1)
    last_values_seasonal[0, -1] = predicted_seasonal

# Convert the forecast lists to numpy arrays
forecasted_dataset = np.array(forecasted_dataset).reshape(-1, 1)
forecasted_residuals = np.array(forecasted_residuals).reshape(-1, 1)
forecasted_trend = np.array(forecasted_trend).reshape(-1, 1)
forecasted_seasonal = np.array(forecasted_seasonal).reshape(-1, 1)

In [ ]:
# Prepare index for forecast data
forecast_start_index = len(dataset)  # Assuming 'dataset' is your original time series data
forecast_indices = np.arange(forecast_start_index, forecast_start_index + num_future_steps)

# Plot Original Data and Forecasts
plt.figure(figsize=(15, 18))

# Plot Original Dataset
plt.subplot(4, 1, 1)  # 4 rows, 1 column, 1st plot
plt.plot(dataset, label='Original Dataset')
plt.plot(forecast_indices, forecasted_dataset, label='Forecasted Dataset', color='orange')
plt.title('Original Dataset and Forecast')
plt.xlabel('Time')
plt.ylabel('Dataset Values')
plt.legend()

# Plot Residuals
plt.subplot(4, 1, 2)  # 4 rows, 1 column, 2nd plot
plt.plot(residual, label='Original Residuals')  # Assuming 'residual' is your original residuals data
plt.plot(forecast_indices, forecasted_residuals, label='Forecasted Residuals', color='red')
plt.title('Original Residuals and Forecast')
plt.xlabel('Time')
plt.ylabel('Residual Values')
plt.legend()

# Plot Trend
plt.subplot(4, 1, 3)  # 4 rows, 1 column, 3rd plot
plt.plot(trend, label='Original Trend')  # Assuming 'trend' is your original trend data
plt.plot(forecast_indices, forecasted_trend, label='Forecasted Trend', color='green')
plt.title('Original Trend and Forecast')
plt.xlabel('Time')
plt.ylabel('Trend Values')
plt.legend()

# Plot Seasonal
plt.subplot(4, 1, 4)  # 4 rows, 1 column, 4th plot
plt.plot(seasonal, label='Original Seasonal')  # Assuming 'seasonal' is your original seasonal data
plt.plot(forecast_indices, forecasted_seasonal, label='Forecasted Seasonal', color='blue')
plt.title('Original Seasonal and Forecast')
plt.xlabel('Time')
plt.ylabel('Seasonal Values')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
# Combine the trend, residuals, and seasonal additively for forecast
combined_forecast = forecasted_trend + forecasted_residuals + forecasted_seasonal

plt.figure(figsize=(12, 6))
# Assuming 'original_series' is your original time series data
plt.plot(dataset, label='Original Series')

# Plotting the combined forecast
plt.plot(np.arange(len(dataset), len(dataset) + len(combined_forecast)), combined_forecast, label='Combined Forecast', color='orange')

plt.title('Original Time Series and Combined Forecast')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()
plt.show()


In [147]:
model.save_weights('ty_custom_model.h5')

In [1]:
# look_back = 90
# trainX_dataset, trainY_dataset = create_dataset(dataset, look_back)

# # Input for the dataset aspect
# input_dataset = Input(shape=(look_back,))
# net_dataset = Dense(12, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(input_dataset)
# net_dataset = Dense(8, activation='swish', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(net_dataset)
# output_dataset = Dense(1, name='output_dataset')(net_dataset)
# model_dataset = Model(inputs=input_dataset, outputs=output_dataset)
# model_dataset.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# model_dataset.fit([trainX_dataset], trainY_dataset, epochs=8, batch_size=32, verbose=2, shuffle = False)

# # Extract the last 'look_back' values from each aspect
# last_values_dataset = dataset[-look_back:].reshape(1, look_back)

# num_future_steps = 365  # Number of future steps to forecast
# forecasted_values = []

# for _ in range(num_future_steps):
#     # Predict the next value
#     next_step_prediction = model_dataset.predict([last_values_dataset])

#     # Append the predicted value
#     forecasted_values.append(next_step_prediction[0, 0])

#     # Update the main dataset's input for the next prediction
#     last_values_dataset = np.roll(last_values_dataset, -1)
#     last_values_dataset[0, -1] = next_step_prediction

# # Convert forecasted_values to a numpy array
# forecasted_values = np.array(forecasted_values).reshape(-1, 1)

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Plot the original dataset
# plt.figure(figsize=(15, 6))
# plt.plot(dataset, label='Original Dataset')

# # Prepare the forecast data for plotting
# # Starting index for forecast data is right after the end of t/he original dataset
# forecast_start_index = len(dataset)
# # Creating an array for plotting that aligns with the original dataset
# forecastPlot = np.empty((forecast_start_index + num_future_steps, 1))
# forecastPlot[:, :] = np.nan
# # Inserting the forecasted values
# forecastPlot[forecast_start_index:forecast_start_index + num_future_steps, :] = forecasted_values

# # Plotting the forecast
# plt.plot(forecastPlot, label='Forecast (Next 120 days)', color='orange')

# # Adding titles and labels
# plt.title('Original Dataset and 120-Day Forecast')
# plt.xlabel('Time')
# plt.ylabel('Values')
# plt.legend()

# # Show the plot
# plt.show()